### Imports

In [1]:
from ipynb.fs.full._Functions import *

# Variational Quantum Eigensolver

In many real world applications and situations, finding the minimal eigenvalue of a matrix is an important task. In chemistry, the minimal eigenvalue of the Hermitian matrix that describes the behavior of a molecule is the ground state energy of the molecule. Phase estimation is a good candidate to solve this problem in the future, but the current hardware doesn't allow using phase estimation for this purpose.

For this reason, Peruzzo developed the varational quantum eigensolver which is able to estimate the ground state energy of a system using current hardware. 

**The problem**

Given a Hermitian matrix $H$ with a minimal eigenvalue $\lambda_m$ corresponding to an eigenstate $\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$\ket{\phi_m}$, the varational quantum eigensolver allows us to get $\lambda_\theta$ which is an estimate of $\lambda_m$ bounded by 

$$ \lambda_m \leq \lambda_\theta \equiv \bra{\psi_\theta} H \ket{\psi_\theta}, $$

where $\ket{\psi_\theta}$ is the eigenstate corresponding to $\lambda_\theta$. Remember that $\bra{\psi_\theta}H\ket{\psi_\theta}$ denotes the expectation value of $H$ in the state $\ket{\psi_\theta}$. The algorithm applies a circuit $U(\theta)$ to some state $\ket{\psi}$ to get an estimate $U(\theta)\ket{\psi}=\ket{\psi_\theta}$ of $\ket{\psi_m}$. Then, this state is optimized by changing the parameter of $U(\theta)$ in order to minimze the value of $\lambda_\theta$.

**Background**

The variational quantum eigensolver algorithm is just an application of the variational method of quantum mechanics. Some mathematical background is needed to elaborate on this.

The spectral theorem states that the eigenvalues of a Hermitian matrix, $H$, will always be real. Therefore, any eigenvalue $\lambda_i$ will be equal to its complex conjugate, i.e. $\lambda_i = \lambda_i^*$. To facilitate the development of this mathematical background, we will express a Hermitian matrix as

$$ H = \sum_{i=1}^{N} \lambda_i \ket{\psi_i} \bra{\psi_i} $$

Therefore, we can rewrite the expectation value of $H$ as

$$
\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle} 
\begin{align}
    \bra{\psi} H \ket{\psi} &= \bra{\psi} \left(\sum_{i=1}^{N} \lambda_i \ket{\psi_i} \bra{\psi_i} \right) \ket{\psi} \\
    &= \sum_{i=1}^{N} \lambda_i \braket{\psi}{\psi_i} \braket{\psi_i}{\psi} \\
    &= \sum_{i=1}^{N} \lambda_i \left |  \braket{\psi_i}{\psi} \right |^2
\end{align}
$$

This shows that the expectation of a Hermitian matrix can be written as a linear combination using the eigenvalues of $H$. Morever, we know that each term of the combination will be greater than or equal to 0 because $ \left |  \braket{\psi_i}{\psi} \right |^2 \geq 0 $. From this, the equation at the beggining of the notebook 

$$  \lambda_m \leq \bra{\psi} H \ket{\psi} = \sum_{i=1}^{N} \lambda_i \left |  \braket{\psi_i}{\psi} \right |^2 $$

becomes clear and intuitive. This is helpful because we can represent the Hamiltonian that describes a system as a Hermitian matrix, which we can use to determine the ground state of the energy using VQE. We will arbitrarly select an initial state $\ket{\psi}$ (called ansatz) as a guess for $\ket{\psi_m}$, which will improve iteratively using the expectation value $\bra{\psi} H \ket{\psi}$ to update our state.

**Improving the ansatz**

To vary the ansatz in a way that it approaches the desired value, we are going to use a parametrized circuit denoted $U(\theta)$. Applying this circuit to our initial state will yield $U(\theta)\ket{\psi} = \ket{\psi_\theta}$. After many iterations, the expected value $\bra{\psi_\theta} H \ket{\psi_\theta}$ will come closer and closer to $\lambda_m$. The circuit $U(\theta)$ is called a variational form, and it is what we are going to construct. 

When designing the variational form, we need to try to use as few parameters as possible while being able to reach any state in the vector space of our circuit, i.e. $\mathbb{C}^N$ where $N=2^n$ and $n$ is the number of qubits. This ensure that the ability to optimize $\ket{\psi}$ relies solely on the capacity of the classical optimizer and not in the implementation of our variational form.

Let's look at what happens when we only have one qubit, $n=1$. In this case, $U(\theta)$ is simply a $U3$ gate, which we know uses three parameters and looks like

$$ U3(\theta, \phi, \lambda) = \begin{pmatrix} \cos(\theta/2) && -e^{i\lambda}\sin(\theta/2) \\ 
                                               e^{i\phi}\sin(\theta/2) && e^{i\lambda+i\phi}\cos(\theta/2) \end{pmatrix} $$
                                               
We use this gate because we only need three parameters to reach any possible state $\ket{\psi}$. 

**Clasical optimization**

Following the example with $n=1$, we will look at how the parameters are optimized to reach the desired value. The method generally implemented in VQE is Simultaneous Perturbation Stochastic Approximation, abbreviated as [SPSA](https://www.jhuapl.edu/spsa/). This method approximates the gradient of the cost function using only two measurements. It does this by simultaneously perturbing all parameters randomly, in contrast of gradient descent in which each parameter is perturbed one by one. 

**Implementing SPSA**

Suppose we have a differentiable cost function $L(\theta)$ where $\theta$ is a parameter vector and the optimization problem can be expressed as finding an specific $\phi$ such that $\frac{\partial L}{\partial \phi} = 0$. SPSA starts with an initial paramater $\theta_0$. After $k$ iterations of the algorithm, a formula for the $(k+1)$th parameter is 

$$ \theta_{k+1} = \theta_k - a_kg_k(\theta_k), $$

where $a_k$ is an arbitrary positive number and $g_k(\theta_k)$ is defined as the approximate of the derivative of our cost function $g(\theta)=\frac{\partial L}{\partial \phi}$ after $k$ iterations.

SPSA is robust to noise that may occur when measuring our cost function, which is good for us since quantum measurement is not perfect in the currently available machines. Let's see how it does this. Set a function $y(\theta) = L(\theta) + \epsilon$, where $\epsilon$ corresponds to the possible error. Then, we can express our estimate for the gradient (derivative of cost function) as

$$ g_{ki}(\theta_k) = \frac{y(\theta_k + c_k\Delta_k) - y(\theta_k - c_k\Delta_k)}{2c_k\Delta_{ki}}, $$

where $c_k$ is another arbitrary positive number and $\Delta_k = (\Delta_{k1}, \Delta_{k2}, \cdots, \Delta_{kn})^T$ is a perturbation vector. Since $g_k$ and $\Delta_k$ are both vectors, the extra $i$ subindex found in the above formula refers to a specific element of the vector. Morever, we can see that this is the two-sided approximation for a derivative which is defined by 

$$ f'(x) \approx \frac{f(x+\epsilon) - f(x-\epsilon)}{2\epsilon} $$

A better explanation for this and its uses can be found in [Gradient Checking (C2W1L13)](https://www.youtube.com/watch?v=QrzApibhohY) at approximately minute 2:10. Thanks to this, SPSA is robust against noise. It already takes into account a perturbation vector, so the additional shifts that may come from error in measurement won't have a significant impact in the optimization process. 

We can now see the steps to implement the algorithm.

_Step 1._ We pick an initial guess for $\theta_0$ and values for all the arbitrary parameters described above. We define $a_k$ from the first formula as $a_k = a \mathbin{/} (A + k + 1)^\alpha$ and $c_k$ from the second formula as $c_k = c \mathbin{/} (k + 1)^\gamma$. (For more detail on how to pick these values, read section _B. Choice of Gain Sequences $a_k$, $c_k$_ from [3]).

_Step 2._ We generate the perturbation vector $\Delta_k$ by Monte Carlo. Each of the components are generated from a zero-mean probability distribution. Uniform and normal random variables are not allowed. A simple and valid distribution is to use a Bernoulii $\pm 1$ distribution with probability of $1/2$ for each component of $\Delta_k$.

_Step 3._ Evaluate the function $y$ as defined above ($y(\theta) = L(\theta) + \epsilon$) based on perturbation around the current value for $\theta_k$. Two measurements are made: $y(\theta_k + c_k \Delta_k)$ and $y(\theta_k - c_k \Delta_k)$ with the values from the two previous steps. 

_Step 4._ Evaluate the gradient approximation using the formula showed previously. Using the values from the steps above, this would look something like:

$$g_{k}(\theta_k) = \frac{y(\theta + c_k\Delta_k) - y(\theta - c_k\Delta_k)}{2c_k} \begin{bmatrix}\Delta_{k1}^{-1} \\ \Delta_{k2}^{-1} \\ \vdots \\ \Delta_{ki}^{-1} \end{bmatrix}$$

where $\Delta_kn$ is the $n$th component of the $\Delta_k$ vector.

_Step 5._ Update the estimate for $\theta$ using $\theta_{k+1} = \theta_k - a_kg_k(\theta_k)$.

_Step 6._ Repeat from steps 2 through 5 and terminate the algorithm once you get little change in successive iterations

**References**
- [1] https://qiskit.org/textbook/ch-applications/vqe-molecules.html
- [2] https://pennylane.ai/qml/demos/spsa.html
- [3] https://www.jhuapl.edu/spsa/PDF-SPSA/Spall_Implementation_of_the_Simultaneous.PDF